In [ ]:
import numpy as np
import soundfile as sf
from scipy.fft import fft, ifft
import pandas as pd
import statsmodels.api as sm
from scipy.signal import find_peaks
import scipy.signal as sps
import wave

In [4]:
def pitch_finder(data ,  freq):
  auto = sm.tsa.acf(data, nlags=2000)
  peaks = find_peaks(auto)[0] 
  lag = peaks[0] 
  pitch = freq / lag 
  return pitch


In [ ]:
def SpeedChanger(const , input , output):
    x, sr = sf.read(input)
    sf.write('temp.wav', x, int(sr*const))
    data , freq = sf.read('temp.wav')
    pitch = pitch_finder(x[:, 0] , freq)
    new_pitch = pitch_finder(data[:, 0] , freq * const)
    shift_amount = abs( new_pitch - pitch)
    shift_right = np.empty(data.shape)
    shift_left = np.empty(data.shape)
    for i in range(0 , len(data)):
      shift_right[i] = data[i] *(np.cos(-1*shift_amount*2*np.pi *i) + 1j*np.sin(-1*shift_amount*i*2*np.pi)) 
      shift_left[i] = data[i] *(np.cos(2*shift_amount*2*np.pi *i) + 1j*np.sin(2*shift_amount*i*2*np.pi)) #pow(np.exp, 1j * shift_amount *i)
    result = np.concatenate((shift_left,shift_right))
    sf.write(output, result, freq)

SpeedChanger( 2, 'input.wav' , 'output.wav')

